# Neural Network with JAX

In this lab, we will:
1. Downloads and loads MNIST into NumPy arrays (if it doesn't already
   exist locally).
2. Builds a simple Multi-Layer Perceptron in JAX.
3. Trains the network on MNIST.
4. Evaluates the performance on test data.
5. Provides a custom inference function for your own handwriting
   images.

This lab is based on a
[JAX example](https://github.com/jax-ml/jax/blob/main/examples/mnist_classifier_fromscratch.py).

Please notice that MNIST is the "hello world" for machine learning,
and there are many many examples available online, including some
simplier ones that use libraries:
[JAX with pre-built optimizers](https://github.com/jax-ml/jax/blob/main/examples/mnist_classifier.py),
[FLAX](https://flax.readthedocs.io/en/latest/mnist_tutorial.html),
[pytorch](https://github.com/pytorch/examples/tree/main/mnist), and
[Keras](https://www.tensorflow.org/datasets/keras_example).

## MNIST Data Loader

We start by downloading the MNIST data set and store it locally.
Our data loader will parse, reshape, normalize them, and return them
in NumPy arrays.

In [ ]:
from os.path import isfile
from urllib.request import urlretrieve

base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

# File names
files = {
    "train_images": "train-images-idx3-ubyte.gz",
    "train_labels": "train-labels-idx1-ubyte.gz",
    "test_images":  "t10k-images-idx3-ubyte.gz",
    "test_labels":  "t10k-labels-idx1-ubyte.gz",
}

for key, file in files.items():
    if not isfile(file):
        url = base_url + file
        print(f"Downloading {url} to {file}...")
        urlretrieve(url, file)
    else:
        print(f"{file} exists; skip download")

In [ ]:
import gzip
import struct
import array
from jax import numpy as np

# Parsing functions

def parse_labels(file):
    with gzip.open(file, "rb") as fh:
        _magic, num_data = struct.unpack(">II", fh.read(8))
        # Read the label data as 1-byte unsigned integers
        return np.array(array.array("B", fh.read()), dtype=np.uint8)

def parse_images(file):
    with gzip.open(file, "rb") as fh:
        _magic, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
        # Read the image data as 1-byte unsigned integers
        images = np.array(array.array("B", fh.read()), dtype=np.uint8)
        # Reshape to (num_data, 28, 28)
        images = images.reshape(num_data, rows, cols)
        return images

In [ ]:
# Parse raw data

train_images_raw = parse_images(files["train_images"])
train_labels_raw = parse_labels(files["train_labels"])

test_images_raw  = parse_images(files["test_images"])
test_labels_raw  = parse_labels(files["test_labels"])

In [ ]:
# Standardize the images, i.e., flatten and normalize images to [0, 1]
def standardize(images):
    return images.reshape(-1, 28*28).astype(np.float32) / 255

train_images = standardize(train_images_raw)
test_images  = standardize(test_images_raw)

In [ ]:
# One-hot encode labels
def one_hot(labels, num_classes=10):
    return np.eye(num_classes)[labels]

train_labels = one_hot(train_labels_raw, 10).astype(np.float32)
test_labels  = one_hot(test_labels_raw,  10).astype(np.float32)

## Visualize Some Training and Testing Data

Let's take a look at our data set!

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(train_images_raw[0,:,:], cmap='gray')

In [ ]:
plt.imshow(test_images_raw[0,:,:], cmap='gray')

## Define a Simple Neural Network in JAX

In this subsection, we introduce the core function needed to
initialize the parameters of a multi-layer network.
Our network will have multiple layers, each characterized by a weight
matrix `W` and a bias vector `b`.
We will use random initialization scaled by a small factor to ensure
stable starting values for training.

In [ ]:
from numpy.random import RandomState

def init_params(scale, layer_sizes, rng=RandomState(0)):
    """
    Initialize the parameters (weights and biases) for each layer in the network.

    Parameters
    ----------
    scale : float
        A scaling factor to control the initial range of the weights.
    layer_sizes : list of int
        The sizes of each layer in the network.
        e.g., [784, 1024, 1024, 10] means:
            - Input layer: 784 units
            - Hidden layer 1: 1024 units
            - Hidden layer 2: 1024 units
            - Output layer: 10 units
    rng : numpy.random.RandomState
        Random state for reproducibility.

    Returns
    -------
    params : list of tuples (W, b)
        Each tuple contains (W, b) for a layer.
        - W is a (input_dim, output_dim) array of weights
        - b is a (output_dim,) array of biases
    """
    return [
        (scale * rng.randn(m, n), scale * rng.randn(n))
        for m, n in zip(layer_sizes[:-1], layer_sizes[1:])
    ]

In the above function,
* We specify a list of layer sizes: for example,
  `[784, 1024, 1024, 10]`.
* For each pair of consecutive sizes `(m, n)`, we create a weight
  matrix W of shape `(m, n)` and a bias vector `b` of shape `(n,)`.
* Multiplying by scale ensures that initial values are not too large,
  which helps prevent numerical issues early in training.
* We store all `(W, b)` pairs in a list, one pair per layer, to be
  used throughout training and inference.

By calling `init_params(scale, layer_sizes)`, you obtain an
easy-to-manipulate structure that keeps all the parameters needed for
your neural network.


In [ ]:
# Define network architecture and hyperparameters

layer_sizes = [784, 1024, 1024, 10]  # 2 hidden layers
param_scale = 0.1

In [ ]:
# Initialize parameters

params = init_params(param_scale, layer_sizes)

## Forward Pass: The `predict` Function

Once the network parameters are initialized, we need a function to
perform the forward pass, producing an output for each batch of
inputs.
Below, we define `predict` to process data through multiple layers,
using a `tanh` activation on the hidden layers, and compute a
log-softmax on the final output layer for stability.

In [ ]:
from jax import numpy as np
from jax.scipy.special import logsumexp

In [ ]:
def predict(params, inputs):
    """
    Compute the network's output logits for a batch of inputs, then subtract
    log-sum-exp for numerical stability (log-softmax).

    Network architecture:
      - Hidden layers use tanh activation
      - Output layer is linear (we'll do log-softmax here)

    Parameters
    ----------
    params : list of (W, b) tuples
        Network's parameters for each layer.
    inputs : np.ndarray
        A batch of input data of shape (batch_size, input_dim).

    Returns
    -------
    np.ndarray
        Log probabilities of shape (batch_size, 10).
    """
    activations = inputs

    # Hidden layers
    for w, b in params[:-1]:
        outputs = np.dot(activations, w) + b
        activations = np.tanh(outputs)

    # Final layer (logits)
    final_w, final_b = params[-1]
    logits = np.dot(activations, final_w) + final_b

    # Log-Softmax: subtract logsumexp for numerical stability
    return logits - logsumexp(logits, axis=1, keepdims=True)

In the above function,
* Hidden Layers (`tanh`):
  Each hidden layer applies a linear transformation
  (`np.dot(activations, w) + b`) followed by the hyperbolic tangent
  activation function (`np.tanh`).
* Final Layer (`logits`):
  The last layer's output is not activated by tanh; instead, we use it
  directly as logits.
* Log-Softmax:
  We transform logits to log probabilities by subtracting the
  logsumexp(logits) along the class dimension.
  This step ensures numerical stability and can be directly used to
  compute losses like cross-entropy.

## Defining the Loss Function

To guide training, we need a loss function that measures how well our
network's predictions match the true labels.
This is like $\chi^2$ when we need to fit a curve.
Below, we define a function that computes the negative log-likelihood
(NLL) over a batch of data.

In [ ]:
def loss(params, batch):
    """
    Computes the average negative log-likelihood loss for a batch.

    Parameters
    ----------
    params : list of (W, b) tuples
        The network's parameters.
    batch : tuple (inputs, targets)
        - inputs: np.ndarray of shape (batch_size, 784)
        - targets: np.ndarray of shape (batch_size, 10) (one-hot labels)

    Returns
    -------
    float
        Mean negative log-likelihood over the batch.
    """
    inputs, targets = batch
    preds = predict(params, inputs)
    
    # preds are log-probs, multiply with one-hot targets and sum -> log-likelihood
    return -np.mean(np.sum(preds * targets, axis=1))

* Inputs and Targets:
  A single batch typically consists of a set of input vectors (inputs)
  and corresponding one-hot encoded labels (targets).
* Forward Pass:
  We call predict(params, inputs), which returns the log probabilities
  for each class.
* NLL Computation:
  We multiply the log probabilities by the one-hot labels (so we only
  pick out the log probability of the correct class for each example).
  Summing these values (log-likelihood) and then negating yields the
  negative log-likelihood.
* Mean Value:
  We take the average across the batch, yielding a scalar loss.

This loss metric drives parameter updates: minimizing it pushes the
network to assign higher probabilities to the correct classes.

## Evaluating Model Performance

While the network is trained by minimizing the negative log-likelihood
(NLL), we often monitor accuracy to get an intuitive sense of model
performance.
The function below calculates the fraction of samples in a batch that
are correctly classified.

In [ ]:
def accuracy(params, batch):
    """
    Computes classification accuracy of the network on a given batch.

    Parameters
    ----------
    params : list of (W, b) tuples
        The network's parameters.
    batch : tuple (inputs, targets)
        - inputs: np.ndarray (batch_size, 784)
        - targets: np.ndarray (batch_size, 10) (one-hot labels)

    Returns
    -------
    float
        Fraction of correctly classified samples.
    """
    inputs, targets = batch
    target_class = np.argmax(targets, axis=1)  # ground truth index
    predicted_class = np.argmax(predict(params, inputs), axis=1)
    return np.mean(predicted_class == target_class)

* Predicted Class:
  * We use predict(params, inputs) to get log probabilities.
  * Taking the argmax across the class dimension finds the class with
    the highest log probability.
* Compare to Ground Truth:
  * We similarly get the ground truth label indices from the one-hot
    targets by using np.argmax(targets, axis=1).
* Accuracy Computation:
  * We compute the fraction of instances where the predicted class
    matches the ground-truth class.
  * This value ranges between 0 (no correct predictions) and 1
    (perfect classification).

Monitoring accuracy alongside the loss offers a more intuitive measure
of how well the model performs on a classification task.

## Gradient Descent for Training: JIT-Compiled Update Function

To optimize our network, we can use Stochastic Gradient Descent (SGD),
updating parameters in the direction that reduces the loss.
This is essentially the same algorithm we implemented in our
optimization class!
Except we only implement a single step for now.
Here, we decorate our update step with `@jit` to compile it for
efficient execution on CPU or GPU.

In [ ]:
from jax import jit, grad

@jit
def update(params, batch, step_size):
    """
    Single step of gradient-based parameter update using simple SGD.

    grad(loss)(params, batch) computes the gradient of the loss function
    with respect to the parameters for the given batch.
    """
    grads = grad(loss)(params, batch)
    return [
        (w - step_size * dw, b - step_size * db)
        for (w, b), (dw, db) in zip(params, grads)
    ]

* `grad(loss)` automatically differentiates the loss function with
  respect to all parameters (`params`), i.e., parameter gradients.
* SGD Update:
  * For each weight `w`, we update it by `w - step_size * dw`.
  * Similarly for each bias `b`.
* `@jit` decorator:
  * Compiles the update step using XLA (Accelerated Linear Algebra).
  * Improves performance by running the update efficiently on CPU/GPU.